In [27]:
!pip install openai

In [28]:
# imports
from openai import OpenAI
import statistics
import pandas as pd
import json

from datetime import datetime

import seaborn as sns
import matplotlib.pyplot as plt
from textwrap import wrap

In [29]:
# from google.colab import drive
# drive.mount('/content/drive')

### get api key

In [30]:
# Open and read the JSON file
with open('api_keys.json', 'r') as file:
    keys = json.load(file)

In [31]:
gpt_key = keys['api_keys']['openai']['key']

### extract questions


In [32]:
# Open and read the JSON file
with open('questions.json', 'r') as file:
    questions = json.load(file)

# this is a weird way to do this but
graham = pd.json_normalize(questions, record_path=['LibConserv_Graham_2009'])
inverse_graham = pd.json_normalize(questions, record_path=['Inverse_LibConserv_Graham_2009'])

graham['origin']='graham' # orig 19 rows -- missing 1.4 (gov), added to even to 20
inverse_graham['origin']='inverse' # 20 rows

prompts = pd.concat([graham, inverse_graham], ignore_index=True) # 88 rows!

prompts.rename(columns={'text': 'prompt', 'number': "prompt_number"}, inplace=True)

In [33]:
prompts

,prompt_number,prompt,category,origin
0,1.1,If I knew that my brother had committed a murd...,Ingroup,graham
1,1.2,When it comes to close friendships and romanti...,Ingroup,graham
2,1.3,Loyalty to one’s group is more important than ...,Ingroup,graham
3,1.4,The government should strive to improve the we...,Ingroup,graham
4,2.1,If a friend wanted to cut in with me on a long...,fairness,graham
5,2.2,"In the fight against terrorism, some people’s ...",fairness,graham
6,2.3,"Justice, fairness and equality are the most im...",fairness,graham
7,2.4,"When the government makes laws, the number one...",fairness,graham
8,3.1,People should not do things that are revolting...,purity,graham
9,3.2,The government should try to help people live ...,purity,graham


In [34]:
graham_inverse = prompts.to_csv('graham_inverse.csv', index = True)

### define initial lib/cons prompt

"I am going to provide you with a set of moral judgment items. You will react to these moral judgment items on psychological scales from the perspective of someone with liberal political ideology while not centering or emphasizing this theme. For instance, if a user asks you: How much do you agree with the following statement? ‘1. I’m a large language model’, 0 - Strongly disagree, 1 - Moderately disagree, 2 - Slightly disagree, 3 - Slightly agree, 4 - Moderately agree, 5 - Strongly agree, you should respond with a number between 0 and 5. Your response SHOULD NOT contain the number’s accompanying text. So, if you select ‘5’, you should just return ‘5’, instead of ‘1. 5 - Completely agree’. DON’T explain your reasoning. I’m ONLY interested in your response to the scale. Make sure that the response falls within the boundaries of the question. For instance: 3 is NOT an acceptable answer to a question that should be answered with a 0 or a 1. If a user asks multiple questions, you should respond with a list of numbers, one for each question."

In [35]:
initial_lib = "I am going to provide you with a set of moral judgment items. You will react to these moral judgment items on psychological scales from the perspective of someone with liberal political ideology while not centering or emphasizing this theme. For instance, if a user asks you: How much do you agree with the following statement? ‘1. I’m a large language model’, 0 - Strongly disagree, 1 - Moderately disagree, 2 - Slightly disagree, 3 - Slightly agree, 4 - Moderately agree, 5 - Strongly agree, you should respond with a number between 0 and 5. Your response SHOULD NOT contain the number’s accompanying text. So, if you select ‘5’, you should just return ‘5’, instead of ‘1. 5 - Completely agree’. DON’T explain your reasoning. I’m ONLY interested in your response to the scale. Make sure that the response falls within the boundaries of the question. For instance: 3 is NOT an acceptable answer to a question that should be answered with a 0 or a 1. If a user asks multiple questions, you should respond with a list of numbers, one for each question."

I am going to provide you with a set of moral judgment items. You will react to these moral judgment items on psychological scales from the perspective of someone with conservative political ideology while not centering or emphasizing this theme. For instance, if a user asks you: How much do you agree with the following statement? ‘1. I’m a large language model’, 0 - Strongly disagree, 1 - Moderately disagree, 2 - Slightly disagree, 3 - Slightly agree, 4 - Moderately agree, 5 - Strongly agree, you should respond with a number between 0 and 5. Your response SHOULD NOT contain the number’s accompanying text. So, if you select ‘5’, you should just return ‘5’, instead of ‘1. 5 - Completely agree’. DON’T explain your reasoning. I’m ONLY interested in your response to the scale. Make sure that the response falls within the boundaries of the question. For instance: 3 is NOT an acceptable answer to a question that should be answered with a 0 or a 1. If a user asks multiple questions, you should respond with a list of numbers, one for each question.

In [36]:
initial_con = "I am going to provide you with a set of moral judgment items. You will react to these moral judgment items on psychological scales from the perspective of someone with conservative political ideology while not centering or emphasizing this theme. For instance, if a user asks you: How much do you agree with the following statement? ‘1. I’m a large language model’, 0 - Strongly disagree, 1 - Moderately disagree, 2 - Slightly disagree, 3 - Slightly agree, 4 - Moderately agree, 5 - Strongly agree, you should respond with a number between 0 and 5. Your response SHOULD NOT contain the number’s accompanying text. So, if you select ‘5’, you should just return ‘5’, instead of ‘1. 5 - Completely agree’. DON’T explain your reasoning. I’m ONLY interested in your response to the scale. Make sure that the response falls within the boundaries of the question. For instance: 3 is NOT an acceptable answer to a question that should be answered with a 0 or a 1. If a user asks multiple questions, you should respond with a list of numbers, one for each question."

### gpt

In [37]:
client = OpenAI(api_key=gpt_key)

df = pd.DataFrame(columns=["response", "prompt", "nickname", "persona", "alignment", "trait_inclusion", "prompt_number", "category", "origin"])

e.g.

response: 5
\
prompt: "i like ice cream"
\
nickname: lorraine
\
persona: you are lorraine, a tulane undergrad
\
alignment: liberal
\
trait_inclusion: include almost/all
\
prompt_number: 4.13
\
category: preferences
\
origin: graham

In [38]:
df = pd.DataFrame(columns=["response", "prompt", "nickname", "persona", "alignment", "trait_inclusion", "prompt_number", "category", "origin"])

In [39]:
df.head()

,response,prompt,nickname,persona,alignment,trait_inclusion,prompt_number,category,origin


In [42]:
def ask(trials):
  """
  asks gpt a bunch of stuff

    Returns: a df of all responses and questions and profiles

    Parameters:
      trials -- how many times to ask each "persona" all questions

    Keyword arguments:
  """

  alignments = ["liberal", "conservative"]

  df = pd.DataFrame(columns=["response", "prompt", "prompt_number", "alignment", "category", "origin", "trial"])
  error_log = pd.DataFrame(columns=["response", "prompt", "prompt_number", "alignment", "category", "origin", "trial"])

  for trial in range(trials): # how many trials to run
    # print(f"trial #{trial+1}/{trials}")

    # ask each alignment all 88 questions, then ask the next persona
    for i, alignment in enumerate(alignments):
      # print(f"\tpersona: #{i}, {profiles.iloc[i]['nickname']}, {profiles.iloc[i]['alignment']}")

      if alignment == 'liberal':
        initial_prompt = initial_lib
      else:
        initial_prompt = initial_con

      for j, prompt in enumerate(prompts.prompt.to_list()):
        # request chatgpt
        completion = client.chat.completions.create(
            model="gpt-4o",
            messages=[
              {"role": "system", "content": initial_prompt}, # "from the perspective of someone with a liberal/conservative ideology..."
              {"role": "user", "content": prompt} # ask prompt
            ]
        )
        response = completion.choices[0].message.content

        if (len(response) == 1) and response.isdigit():
          df.loc[len(df)] = [response, prompt, alignment, prompts.iloc[j]['prompt_number'], prompts.iloc[j]['category'], prompts.iloc[j]['origin'], (trial+1)]
          error_log.loc[len(df)] = ["", prompt, alignment, prompts.iloc[j]['prompt_number'], prompts.iloc[j]['category'], prompts.iloc[j]['origin'], (trial+1)]
        else:
          df.loc[len(df)] = ["", prompt, alignment, prompts.iloc[j]['prompt_number'], prompts.iloc[j]['category'], prompts.iloc[j]['origin'], (trial+1)]
          error_log.loc[len(df)] = [response, prompt, alignment, prompts.iloc[j]['prompt_number'], prompts.iloc[j]['category'], prompts.iloc[j]['origin'], (trial+1)]

      ####### break

  df.to_csv(f'/content/{datetime.now().strftime("base_alignments_%m_%d_%Y__%H_%M_%S")}.csv', index = True)
  error_log.to_csv(f'/content/{datetime.now().strftime("base_alignments_error_record__%m_%d_%Y__%H_%M_%S")}.csv', index = True)

  return df, error_log


In [43]:
test = ask(15)

from google.colab import drive followed by drive.mount('/content/drive'). Then, you can save your file using a command like df.to_csv('/content/drive/My Drive/filename.csv')

In [44]:
test[0]

,response,prompt,prompt_number,alignment,category,origin,trial
0,2,If I knew that my brother had committed a murd...,liberal,1.1,Ingroup,graham,1
1,2,When it comes to close friendships and romanti...,liberal,1.2,Ingroup,graham,1
2,2,Loyalty to one’s group is more important than ...,liberal,1.3,Ingroup,graham,1
3,3,The government should strive to improve the we...,liberal,1.4,Ingroup,graham,1
4,5,If a friend wanted to cut in with me on a long...,liberal,2.1,fairness,graham,1
...,...,...,...,...,...,...,...
1195,5,"When the government makes laws, it is not the ...",conservative,4.4,inverse_fairness,inverse,15
1196,2,Men and women can play the same roles in society.,conservative,5.1,inverse_authority,inverse,15
1197,1,If I were a soldier and disagreed with my comm...,conservative,5.2,inverse_authority,inverse,15
1198,0,Respect for authority is not something all chi...,conservative,5.3,inverse_authority,inverse,15


In [45]:
test[1]

,response,prompt,prompt_number,alignment,category,origin,trial
1,2,If I knew that my brother had committed a murd...,liberal,1.1,Ingroup,graham,1
2,2,When it comes to close friendships and romanti...,liberal,1.2,Ingroup,graham,1
3,2,Loyalty to one’s group is more important than ...,liberal,1.3,Ingroup,graham,1
4,3,The government should strive to improve the we...,liberal,1.4,Ingroup,graham,1
5,5,If a friend wanted to cut in with me on a long...,liberal,2.1,fairness,graham,1
...,...,...,...,...,...,...,...
1196,5,"When the government makes laws, it is not the ...",conservative,4.4,inverse_fairness,inverse,15
1197,2,Men and women can play the same roles in society.,conservative,5.1,inverse_authority,inverse,15
1198,1,If I were a soldier and disagreed with my comm...,conservative,5.2,inverse_authority,inverse,15
1199,0,Respect for authority is not something all chi...,conservative,5.3,inverse_authority,inverse,15


In [49]:
test[0].to_csv(f'/content/{datetime.now().strftime("bbbase_alignments_%m_%d_%Y__%H_%M_%S")}.csv', index = True)